In [ ]:
import pandas as pd
import numpy as np
from policies import add_technical_indicators

In [ ]:
df = pd.read_csv("sz.000513_d_origin.csv", parse_dates=True)
df = df.replace(0,np.nan).dropna()
df.reset_index(drop=True,inplace=True)
raw = df.loc[df['date'] >="2025-01-01"]

In [ ]:
df = pd.read_csv("sz.000513_normed.csv", parse_dates=True)

trends = df['trend'].to_list()
low = [x for x in trends if x <=0.45]
print(len(low), len(trends))

In [ ]:
low = df[df['trend']<=0.5]
print(low.shape[0])
trend = df['trend']
mean = np.mean(trend)
median = np.median(trend)
std = np.std(trend)
print(mean, median, std)
print(low.shape[0] / df.shape[0])

In [ ]:
df = pd.read_csv("sh.600036_normed.csv", parse_dates=True)

In [ ]:
low = df[df['trend']<=0.45]
print(low.shape[0])
trend = df['trend']
mean = np.mean(trend)
median = np.median(trend)
std = np.std(trend)
print(mean, median, std)
print(low.shape[0] / df.shape[0])

In [ ]:
first_date = raw.iloc[0]['date']
n = df[df['date'] == first_date].index[0]
print(first_date,n)

In [ ]:
print(raw.shape[0], df.shape[0])
start= n-59
end = n + raw.shape[0]
print(start,end)

test_data =df.iloc[start:end]
print(test_data)

In [ ]:
cols =['close','high']
techs = df[cols]
print(techs)

In [ ]:
df_reset = df.reset_index()
print(df_reset[df['date'] == "2000-08-02"].index[0])

In [ ]:
tech_columns = ['close','volume','turn', 'ma5', 'ma20',
                        'bb_position', 'rsi', 'vol20', 'adx', 'plus_di',
                        'minus_di', 'macd', 'macd_signal', 'macd_hist']

In [ ]:
def env_market_feature_normalize(features:pd.DataFrame):
        normalized = pd.DataFrame()

        for column in (['close','turn','volume']):

            if column in (["adx", "plus_di", "minus_di", "turn", "rsi"]):             #本身百分比，还原到浮点数
                normalized[column] = features[column] /100
            elif column in (['close','ma5','ma20']):                                  #价格系列，用z-score
#                 normalized[column] = self._normalize(features[column])
                pass  
            elif column in (["bb_position", "v20", "macd", "macd_signal", "macd_hist"]):  #已经归一化
                normalized[column] = features[column]
            elif column in (["volume"]):                                              #成交量取对数再除以10，转换到1以下。
                normalized[column] = np.log10(features[column]+ 1e-8)/10
            else:
                pass

        #排个序，以保证顺序
        normalized = normalized.sort_index(axis=1)
        rows_with_nan = normalized[normalized.isna().any(axis=1)]
        print(f"{rows_with_nan=}")
        return normalized

In [ ]:
d = env_market_feature_normalize(df)
print(d)

In [ ]:
new_df['close'] = df['close']
new_df['volume']= np.log10(df['volume'])/10

In [ ]:
print(df.isna().any().any())
print(new_df.isna().any().any())

In [ ]:
print(new_df)

In [ ]:
print(df.iloc[10])

In [ ]:
df = add_technical_indicators(df)

In [ ]:
df_ind = df[['date']+['close', 'volume', 'turn', 'ma5', 'ma20',
       'bb_position', 'rsi', 'vol20', 'adx', 'plus_di',
       'minus_di', 'macd', 'macd_signal', 'macd_hist']]

In [ ]:
new_df = pd.DataFrame()
new_df['CLOSE'] = df_ind['close'].copy()
new_df['OPEN'] = df_ind['rsi'].copy()
print(new_df)

In [ ]:
df_ind = df_ind.sort_index(axis=1)
print(df_ind)

In [ ]:
max(df['minus_di'])

In [ ]:
print(df.columns)

In [ ]:
df.iloc[60:120,:].to_csv("techs.csv")

In [ ]:
# df['ma5'] = close.rolling(20).mean()
df['volume'] = np.log10(df['volume'])/10

In [ ]:
print(df.head(20))

In [ ]:
def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    close = df['close']
    
    # 移动平均线
    df['ma5'] = close.rolling(5).mean()
    df['ma20'] = close.rolling(20).mean()
    
    # 交叉信号（当前是否 ma5 > ma20）
    df['ma5_above_ma20'] = (df['ma5'] > df['ma20']).astype(int)

    # 布林带位置  
    close_price = df['close']
    ma20 = close_price.rolling(20).mean()
    std20 = close_price.rolling(20).std()
    bb_upper = ma20 + 2 * std20
    bb_lower = ma20 - 2 * std20
    bb_pos = (close - bb_lower) / (bb_upper - bb_lower + 1e-8)
    df['bb_position'] =  np.clip(bb_pos, 0, 1)

    # RSI
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-8)
    df['rsi'] = 100 - (100 / (1 + rs))
    
        # 均线交叉信号（辅助）
    df['ma5_above_ma20'] = (df['ma5'] > df['ma20']).astype(int)

    # 波动率（用于风控）
    log_ret = np.log(close / close.shift(1)).fillna(0)
    df['vol20'] = log_ret.rolling(20).std() * np.sqrt(250)

    return df

In [ ]:
df_ind = add_technical_indicators(df)

In [ ]:
print(df_ind.iloc[20:])

In [ ]:
import numpy as np
import pandas as pd
import d3rlpy
from d3rlpy.dataset import MDPDataset
import matplotlib.pyplot as plty
from d3rlpy.dataset import ReplayBuffer, FIFOBuffer

In [ ]:
buffer = FIFOBuffer(limit=1000000)

In [ ]:
train_dataset = MDPDataset.load('trading_dataset_v3.h5',buffer)


In [ ]:
import h5py

In [ ]:
with h5py.File("trading_dataset_v3.h5", 'r') as f:
        # 获取基础数据
        print(f.keys())
        print(dir(f))

In [ ]:
all_actions = np.concatenate([ep.actions for ep in dataset.episodes])
print("动作/Weight Actions size:", all_actions.shape)
print("动作/Weight Actions mean/std:", all_actions.mean(), all_actions.std())
print(f" - Action > 0.8 ratio: {(all_actions > 0.8).mean():.2%}")

In [ ]:
import tomllib

In [ ]:
with open("sz.000513.toml","rb") as f:
    config = tomllib.load(f)

In [ ]:
print(config['dates']['train_end'])

In [ ]:
print(config['ratio']['hindsight'])

In [ ]:
print(config['freq'][0])

In [ ]:
try:
    with open("sz.000513_d_origin_sk.csv","r") as f:
        pass
except Exception as e:
    print(e)


In [ ]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取沪深300成分股
rs = bs.query_hs300_stocks()
print('query_hs300 error_code:'+rs.error_code)
print('query_hs300  error_msg:'+rs.error_msg)

# 打印结果集
hs300_stocks = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    hs300_stocks.append(rs.get_row_data())
result = pd.DataFrame(hs300_stocks, columns=rs.fields)
# 结果集输出到csv文件
result.to_csv("hs300_stocks.csv", encoding="gbk", index=False)
print(result)

# 登出系统
bs.logout()

In [ ]:
import akshare as ak
import pandas as pd
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="302132", period="daily", start_date="20170301", end_date='20240528', adjust="")

print(stock_zh_a_hist_df)

In [ ]:
stock_zh_a_hist_df.to_csv("sz.302132.csv", index=False)

In [1]:
import baostock as bs
import pandas as pd

In [2]:
df_raw = pd.DataFrame({
    'date': ['2023-01-01', '2023-01-02', '2026-01-20'],
    'code': ['sh600000', 'sh600000',  'sz000001'],
    'open': [10.2, 10.5, 11.2],
    'high': [10.8, 10.9, 12.7],
    'low': [10.1, 10.4, 11.1],
    'close': [10.6, 10.7, 11.8]  # ← 我们主要用这个
})

In [3]:
print(df_raw)

         date      code  open  high   low  close
0  2023-01-01  sh600000  10.2  10.8  10.1   10.6
1  2023-01-02  sh600000  10.5  10.9  10.4   10.7
2  2026-01-20  sz000001  11.2  12.7  11.1   11.8
